In [1]:
import cv2
import imutils

from pyimagesearch.transform import four_point_transform

In [2]:
class Image:
    def __init__(self, path):
        self.img = cv2.imread(path)
        self.showImage()


    def showImage(self, image=None, time=5000):
        if image is None:
            image = self.img
        cv2.imshow('Image', image)
        cv2.waitKey(time)
        cv2.destroyAllWindows()


    def toGray(self, image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)
        return gray


    def detectEdges(self):
        """
        Detect receipt edges and crop it accordingly.
        """
        resized = imutils.resize(self.img, height = 500)
        ratio = self.img.shape[0] / 500.0
        gray = self.toGray(resized)
        edged = cv2.Canny(gray, 75, 200)

        cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
        
        for c in cnts:
            # approximate the contour
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)

            # if our approximated contour has four points, then we
            # can assume that we have found our screen
            if len(approx) == 4:
                screenCnt = approx
                break

        warped = four_point_transform(self.img, screenCnt.reshape(4, 2) * ratio)
        self.showImage(imutils.resize(warped, height = 650))

In [3]:
warped = Image('images/example.jpg')

In [4]:
warped.detectEdges()